# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [25]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
from pymongo import UpdateOne
from decimal import Decimal


In [3]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [5]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'config', 'local']


In [7]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [8]:
# review the collections in our new database
print(db.list_collection_names())

[]


In [9]:
# review a document in the establishments collection
pprint(db.establishments.find_one())

None


In [11]:
# assign the collection to a variable
establishments = db['establishments']
pprint(establishments.find_one())


None


## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [12]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "BusinessName": "The Coding Café",
    "AddressLine1": "123 Code Street",
    "AddressLine2": "Suite 101",
    "AddressLine3": "",
    "AddressLine4": "",
    "PostCode": "AB12 3CD",
    "FoodType": "Cafe",
    "RatingValue": 5,
    "RatingDate": "2024-11-30",
    "LocalAuthority": "Tech Borough",
    "Geocode": {
        "Latitude": 51.5074,
        "Longitude": -0.1278
    }
}

pprint(new_restaurant)


{'AddressLine1': '123 Code Street',
 'AddressLine2': 'Suite 101',
 'AddressLine3': '',
 'AddressLine4': '',
 'BusinessName': 'The Coding Café',
 'FoodType': 'Cafe',
 'Geocode': {'Latitude': 51.5074, 'Longitude': -0.1278},
 'LocalAuthority': 'Tech Borough',
 'PostCode': 'AB12 3CD',
 'RatingDate': '2024-11-30',
 'RatingValue': 5}


In [13]:
# Insert the new restaurant into the collection
insert_result = establishments.insert_one(new_restaurant)

print(f"New restaurant inserted with ID: {insert_result.inserted_id}")


New restaurant inserted with ID: 674b5d924bc660c500c5a01f


In [15]:
# Check that the new restaurant was inserted
result = establishments.find_one({"BusinessName": "The Coding Café"})
result


{'_id': ObjectId('674b5d924bc660c500c5a01f'),
 'BusinessName': 'The Coding Café',
 'AddressLine1': '123 Code Street',
 'AddressLine2': 'Suite 101',
 'AddressLine3': '',
 'AddressLine4': '',
 'PostCode': 'AB12 3CD',
 'FoodType': 'Cafe',
 'RatingValue': 5,
 'RatingDate': '2024-11-30',
 'LocalAuthority': 'Tech Borough',
 'Geocode': {'Latitude': 51.5074, 'Longitude': -0.1278}}

2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [16]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
result = establishments.find_one(
    {"BusinessType": "Restaurant/Cafe/Canteen"},
    {"BusinessTypeID": 1, "BusinessType": 1, "_id": 0}
)
pprint(result)


None


3. Update the new restaurant with the `BusinessTypeID` you found.

In [17]:
# Update the new restaurant with the correct BusinessTypeID
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen"
business_type = establishments.find_one(
    {"BusinessType": "Restaurant/Cafe/Canteen"},
    {"BusinessTypeID": 1, "_id": 0}
)

if business_type:
    result = establishments.update_one(
        {"BusinessName": "The Coding Café"},  
        {"$set": {"BusinessTypeID": business_type["BusinessTypeID"]}} 
    )
    print(f"Matched {result.matched_count} document(s), updated {result.modified_count} document(s).")
else:
    print("BusinessTypeID for 'Restaurant/Cafe/Canteen' not found.")


BusinessTypeID for 'Restaurant/Cafe/Canteen' not found.


In [18]:
# Confirm that the new restaurant was updated
updated_restaurant = establishments.find_one({"BusinessName": "The Coding Café"})
pprint(updated_restaurant)


{'AddressLine1': '123 Code Street',
 'AddressLine2': 'Suite 101',
 'AddressLine3': '',
 'AddressLine4': '',
 'BusinessName': 'The Coding Café',
 'FoodType': 'Cafe',
 'Geocode': {'Latitude': 51.5074, 'Longitude': -0.1278},
 'LocalAuthority': 'Tech Borough',
 'PostCode': 'AB12 3CD',
 'RatingDate': '2024-11-30',
 'RatingValue': 5,
 '_id': ObjectId('674b5d924bc660c500c5a01f')}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [20]:
# Find how many documents have LocalAuthorityName as "Dover"
dover_count = establishments.count_documents({"LocalAuthorityName": "Dover"})
dover_count

0

In [21]:
# Delete all documents where LocalAuthorityName is "Dover"
delete_result = establishments.delete_many({"LocalAuthorityName": "Dover"})
print(f"Number of documents deleted: {delete_result.deleted_count}")


Number of documents deleted: 0


In [23]:
# Check if any remaining documents include Dover
remaining_dover = establishments.find_one({"LocalAuthorityName": "Dover"})

if remaining_dover:
    print("There are still documents with LocalAuthorityName as 'Dover'.")
    pprint(remaining_dover)
else:
    print("No documents with LocalAuthorityName as 'Dover' remain.")


No documents with LocalAuthorityName as 'Dover' remain.


In [24]:
# Check that other documents remain with 'find_one'
remaining_document = establishments.find_one()

if remaining_document:
    print("There are still documents in the collection.")
    pprint(remaining_document)
else:
    print("No documents remain in the collection.")


There are still documents in the collection.
{'AddressLine1': '123 Code Street',
 'AddressLine2': 'Suite 101',
 'AddressLine3': '',
 'AddressLine4': '',
 'BusinessName': 'The Coding Café',
 'FoodType': 'Cafe',
 'Geocode': {'Latitude': 51.5074, 'Longitude': -0.1278},
 'LocalAuthority': 'Tech Borough',
 'PostCode': 'AB12 3CD',
 'RatingDate': '2024-11-30',
 'RatingValue': 5,
 '_id': ObjectId('674b5d924bc660c500c5a01f')}


5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [26]:
# Change the data type from String to Decimal for longitude and latitude
documents = establishments.find(
    {"Geocode.Latitude": {"$type": "string"}, "Geocode.Longitude": {"$type": "string"}}
)

bulk_operations = []
for doc in documents:
    bulk_operations.append(
        UpdateOne(
            {"_id": doc["_id"]},
            {
                "$set": {
                    "Geocode.Latitude": Decimal(doc["Geocode"]["Latitude"]),
                    "Geocode.Longitude": Decimal(doc["Geocode"]["Longitude"]),
                }
            }
        )
    )

if bulk_operations:
    result = establishments.bulk_write(bulk_operations)
    print(f"Modified {result.modified_count} document(s).")
else:
    print("No documents found with latitude and longitude as strings.")


No documents found with latitude and longitude as strings.


Use `update_many` to convert `RatingValue` to integer numbers.

In [1]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

NameError: name 'establishments' is not defined

In [28]:
# Change the data type from String to Integer for RatingValue
documents = establishments.find({"RatingValue": {"$type": "string"}})

bulk_operations = []
for doc in documents:
    try:
        int_value = int(doc["RatingValue"])
        bulk_operations.append(
            UpdateOne(
                {"_id": doc["_id"]},  
                {"$set": {"RatingValue": int_value}}
            )
        )
    except ValueError:
        pass

if bulk_operations:
    result = establishments.bulk_write(bulk_operations)
    print(f"Modified {result.modified_count} document(s).")
else:
    print("No documents found with RatingValue as a string representing an integer.")


No documents found with RatingValue as a string representing an integer.


In [30]:
# Check that the coordinates and rating value are now numbers
documents = establishments.find({}, {"Geocode.Latitude": 1, "Geocode.Longitude": 1, "RatingValue": 1}).limit(5)

for doc in documents:
    print("Document ID:", doc["_id"])
    print("Latitude:", doc.get("Geocode", {}).get("Latitude"), "Type:", type(doc.get("Geocode", {}).get("Latitude")))
    print("Longitude:", doc.get("Geocode", {}).get("Longitude"), "Type:", type(doc.get("Geocode", {}).get("Longitude")))
    print("RatingValue:", doc.get("RatingValue"), "Type:", type(doc.get("RatingValue")))
    print("-" * 50)


Document ID: 674b5d924bc660c500c5a01f
Latitude: 51.5074 Type: <class 'float'>
Longitude: -0.1278 Type: <class 'float'>
RatingValue: 5 Type: <class 'int'>
--------------------------------------------------
